In [1]:
import pickle
pickle.HIGHEST_PROTOCOL = 4

import numpy as np
import pandas as pd

import os
from pathlib import Path
import func_proc_filepath as mFILE
from tqdm import tqdm
from IPython.display import display
import shutil

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention
from keras.utils import Sequence
from keras import backend as K

import random

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

from sklearn.ensemble import RandomForestClassifier
# LightGBM
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
class F1Callback(Callback):
    def __init__(self, model, X_val, y_val, path):
        self.model = model
        self.X_val = X_val
        self.y_val = y_val
        
        self.path = path
        self.best_score = -1

    def on_epoch_end(self, epoch, logs):
        pred = self.model.predict(self.X_val)
        f1_val = f1_score(self.y_val, np.round(pred), average='macro')
        acc_val = accuracy_score(self.y_val, np.round(pred))
        score = f1_val*0.67 + acc_val*0.33
        log = "Score = {0}, F1 = {1}, ACC = {2}".format(score, f1_val, acc_val)
        print(log)
        # 以下チェックポイントなど必要なら書く
        if score > self.best_score:
            self.best_score = score
            self.model.save(self.path)

In [3]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
# y: after one hot encoding
def create_gru_model(np_train_x, np_train_y, np_val_x, np_val_y, n_class, n_units=64, drop=0.5, lr=1e-4, 
                     batch=128, model_path=""):
    # ** fix random seed **
    FIX_SEED = 49
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(FIX_SEED)
    random.seed(FIX_SEED)
    session_conf = tf.compat.v1.ConfigProto()
    tf.compat.v1.set_random_seed(FIX_SEED)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    # *********************
    
    #*********************************************    
    # モデルの定義
    model = Sequential()
    
    n_rnn = np_train_x.shape[1]
    n_feat = np_train_x.shape[2]
    adam = Adam(lr=lr)
    
    #model.add(Bidirectional(GRU(units=n_units, input_shape=(n_rnn, n_feat), dropout=drop, return_sequences=False)))
    #model.add(GRU(units=128, input_shape=(n_rnn, n_feat), dropout=drop, return_sequences=True))
    #model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(GRU(units=n_units, input_shape=(n_rnn, n_feat), dropout=drop, return_sequences=False))
    model.add(Activation('tanh'))
    model.add(Dropout(drop))
    model.add(Dense(n_class, activation='softmax'))
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy", f1])

    #model_path = dir_out + "\\model_multi_image" + footer + ".h5"
    #cb_early = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    cb_early = EarlyStopping(monitor='val_f1', patience=8, verbose=0, mode='max')
    
    model.fit(np_train_x, np_train_y,
              epochs=30,
              batch_size=batch,
              validation_data=(np_val_x, np_val_y),
              callbacks=[F1Callback(model, np_val_x, np_val_y, model_path), cb_early])
    
    model.summary()
    
    # bestモデルを読み込んで、重みやオプティマイザーを含むモデル全体を再作成
    new_model = keras.models.load_model(model_path, custom_objects={'f1':f1})
    
    pred_nn = new_model.predict(np_val_x)
    score_nn_f = f1_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1), average='macro')
    score_nn_a = accuracy_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1))
    score_nn = score_nn_f*0.67 + score_nn_a*0.33
    
    scores_nn = [score_nn, score_nn_f, score_nn_a]
    
    display(["0.67*F1+0.33*ACC", "F1 score", "ACC score"])
    display(scores_nn)
    
    del model
    del new_model
    
    return scores_nn

In [5]:
# evaluate predict data
def eval_pred_each_class(np_true, np_pred, num_class):

    result = []

    # calc
    for i in range(num_class):
        # i以外を0、iを1に置き換える
        y_true_i = np.where(np_true == i, 1, 0)
        y_pred_i = np.where(np_pred == i, 1, 0)

        recall_i = recall_score(y_true_i, y_pred_i, average='binary')
        precision_i = precision_score(y_true_i, y_pred_i, average='binary')
        f1_i = f1_score(y_true_i, y_pred_i, average='binary')
        acc_i = accuracy_score(y_true_i, y_pred_i)

        df_reslut = pd.DataFrame({"class":[i], "recall":[recall_i], "precision":[precision_i], 
                                 "f1":[f1_i], "accuracy":[acc_i]})
        #result_i = [recall_i, precision_i, f1_i, acc_i]

        result.append(df_reslut)

    df_out = pd.concat([x for x in result], axis=0, ignore_index=True)
    f1_all = df_out.loc[:, "f1"].mean()
    recall_all = df_out.loc[:, "recall"].mean()
    precision_all = df_out.loc[:, "precision"].mean()
    accuracy_all = df_out.loc[:, "accuracy"].mean()
    df_all = pd.DataFrame({"class":["all"], "recall":[recall_all], "precision":[precision_all], 
                                 "f1":[f1_all], "accuracy":[accuracy_all]})
    df_out = pd.concat([df_out, df_all], axis=0, ignore_index=True)

    return df_out


In [6]:
def np_train_balance(np_x, np_y):
    
    ny = np_y.ravel()
    
    np_x0 = np_x[ny==0,:,:]
    np_x0 = np_x0[::2,:,:]
    
    np_y0 = ny[ny==0]
    np_y0 = np_y0[::2]
    
    np_x1 = np_x[ny==1,:,:]
    np_x1 = np.append(np_x1, np_x1, axis=0)
    np_x0 =  np.append(np_x0, np_x1, axis=0)
    np_x1 = None
    
    
    np_y1 = ny[ny==1]
    np_y1 = np.append(np_y1, np_y1)
    np_y0 =  np.append(np_y0, np_y1)
    np_y1 = None
    
    np_x2 = np_x[ny==2,:,:]
    np_x2 = np.append(np_x2, np_x2, axis=0)
    np_x2 = np.append(np_x2, np_x2, axis=0)
    np_x0 =  np.append(np_x0, np_x2, axis=0)
    np_x2 = None
    
    np_y2 = ny[ny==2]
    np_y2 = np.append(np_y2, np_y2)
    np_y2 = np.append(np_y2, np_y2)
    np_y0 =  np.append(np_y0, np_y2)
    np_y2 = None
    
    np_x3 = np_x[ny==3,:,:]
    np_x3 = np.append(np_x3, np_x3, axis=0)
    np_x3 = np.append(np_x3, np_x3, axis=0)
    np_x0 =  np.append(np_x0, np_x3, axis=0)
    np_x3 = None
    
    np_y3 = ny[ny==3]
    np_y3 = np.append(np_y3, np_y3)
    np_y3 = np.append(np_y3, np_y3)
    np_y0 =  np.append(np_y0, np_y3)
    np_y3 = None
    
    np_x4 = np_x[ny==4,:,:]
    np_x4 = np_x4[::3,:,:]
    np_x0 =  np.append(np_x0, np_x4, axis=0)
    np_x4 = None
    
    np_y4 = ny[ny==4]
    np_y4 = np_y4[::3]
    np_y0 =  np.append(np_y0, np_y4)
    np_y4 = None
    
    np_x5 = np_x[ny==5,:,:]
    np_x5 = np_x5[::2,:,:]
    np_x0 =  np.append(np_x0, np_x5, axis=0)
    np_x5 = None
    
    np_y5 = ny[ny==5]
    np_y5 = np_y5[::2]
    np_y0 =  np.append(np_y0, np_y5)
    np_y5 = None
    
    np_x6 = np_x[ny==6,:,:]
    np_x0 =  np.append(np_x0, np_x6, axis=0)
    np_x6 = None
    
    np_y6 = ny[ny==6]
    np_y0 =  np.append(np_y0, np_y6)
    np_y6 = None
    
    np_x = None
    np_y = None
    ny = None
    
    p =np.random.RandomState(seed=49).permutation(len(np_x0))
    np_x0 = np_x0[p]
    np_y0 = np_y0[p]
    
    
    return np_x0, np_y0.reshape(-1,1)

In [7]:
def np_val_balance(np_x, np_y): #::7, 1, *2, 1, ::4, ::3, ::2
    
    ny = np_y.ravel()
    
    np_x0 = np_x[ny==0,:,:]
    np_x0 = np_x0[::3,:,:]
    
    np_y0 = ny[ny==0]
    np_y0 = np_y0[::3]
    
    np_x1 = np_x[ny==1,:,:]
    np_x0 =  np.append(np_x0, np_x1, axis=0)
    np_x1 = None
    
    
    np_y1 = ny[ny==1]
    np_y0 =  np.append(np_y0, np_y1)
    np_y1 = None
    
    np_x2 = np_x[ny==2,:,:]
    np_x2 = np.append(np_x2, np_x2, axis=0)
    np_x0 =  np.append(np_x0, np_x2, axis=0)
    np_x2 = None
    
    np_y2 = ny[ny==2]
    np_y2 = np.append(np_y2, np_y2)
    np_y0 =  np.append(np_y0, np_y2)
    np_y2 = None
    
    np_x3 = np_x[ny==3,:,:]
    np_x0 =  np.append(np_x0, np_x3, axis=0)
    np_x3 = None
    
    np_y3 = ny[ny==3]
    np_y0 =  np.append(np_y0, np_y3)
    np_y3 = None
    
    np_x4 = np_x[ny==4,:,:]
    np_x4 = np_x4[::4,:,:]
    np_x0 =  np.append(np_x0, np_x4, axis=0)
    np_x4 = None
    
    np_y4 = ny[ny==4]
    np_y4 = np_y4[::4]
    np_y0 =  np.append(np_y0, np_y4)
    np_y4 = None
    
    np_x5 = np_x[ny==5,:,:]
    np_x5 = np_x5[::3,:,:]
    np_x0 =  np.append(np_x0, np_x5, axis=0)
    np_x5 = None
    
    np_y5 = ny[ny==5]
    np_y5 = np_y5[::3]
    np_y0 =  np.append(np_y0, np_y5)
    np_y5 = None
    
    np_x6 = np_x[ny==6,:,:]
    np_x6 = np_x6[::2,:,:]
    np_x0 =  np.append(np_x0, np_x6, axis=0)
    np_x6 = None
    
    np_y6 = ny[ny==6]
    np_y6 = np_y6[::2]
    np_y0 =  np.append(np_y0, np_y6)
    np_y6 = None
    
    np_x = None
    np_y = None
    ny = None
    
    #p = np.random.permutation(len(np_x0))
    p =np.random.RandomState(seed=49).permutation(len(np_x0))
    np_x0 = np_x0[p]
    np_y0 = np_y0[p]
    
    
    return np_x0, np_y0.reshape(-1,1)

In [9]:
def data_read_separate2(file_list, dir_data, dir_audio, pre_model, transformer, balance=0, sep_num=5, ignore_feat=False):
    
    step = -(-len(file_list) // sep_num)
    
    np_x_out_list = []
    np_y_out_list = []
    
    for s in range(sep_num):
        
        start = step*s
        stop = step*(s+1)
        if stop>len(file_list):
            stop = len(file_list)
            
        file_list_sep = file_list[start:stop]
        
        np_x_list = []
        np_y_list = []

        for i in tqdm(range(len(file_list_sep))):
            name = os.path.splitext(os.path.basename(file_list_sep[i]))[0]

            df_label = pd.read_csv(file_list_sep[i])
            #display(df_label)
            df_label = df_label.drop(["Anger","Disgust","Fear","Happiness","Sadness","Surprise"],axis=1)
            df_label.columns = ["expr"]
            #df_label["frame"] = df_label.index
            df_label.index.name = "frame"

            df_data = pd.read_hdf(dir_data + "\\" + name + ".h5")
            df_data.set_index("frame", drop=True, inplace=True)
            #display(df_data)
            df_audio = pd.read_hdf(dir_audio + "\\" + name + ".h5")
            df_audio.set_index("frame", drop=True, inplace=True)

            #df_merge = pd.merge(df_label, df_data, on="frame", how="outer")
            #df_merge = pd.merge(df_merge, df_audio, on="frame", how="outer")
            df_merge = df_label.join([df_data, df_audio], how="outer")
            # interpolate 30 frame back
            df_merge.interpolate(method="index", limit=30, limit_direction='backward', inplace=True)
            #df_merge.fillna(0, inplace=True)
            #df_merge = df_merge.loc[df_merge.loc[:,"expr"]>=0,:]
            #display(df_merge)

            np_x = df_merge.loc[:, df_merge.columns.str.contains("AU|pose_R|gaze|vgg-")].values
            np_y = df_merge.loc[:, df_merge.columns.str.contains("expr")].values
            pre_x = pre_model.predict(np_x)
            """
            # normalize by single subject
            np_mean_sub = np.nanmean(pre_x, axis=0)
            np_std_sub = np.nanstd(pre_x, axis=0, ddof=1)
            np_x_sub = (pre_x - np_mean_sub)/np_std_sub
            df_sub = pd.DataFrame(np_x_sub)
            df_sub.replace(np.inf, 9999, inplace=True)
            df_sub.fillna(0, inplace=True)
            df_sub.mask(df_sub > 5, 5, inplace=True)
            df_sub.columns = ["sub-" + str(n) for n in df_sub.columns.values]
            # ***************
            """
            df = pd.DataFrame(pre_x)
            #df = pd.concat([df, df_merge.loc[:, df_merge.columns.str.contains("expr")]], axis=1)
            df = pd.concat([df, df_merge.loc[:, df_merge.columns.str.contains("audio")],
                            df_merge.loc[:, df_merge.columns.str.contains("expr")]], axis=1)
            
            batch_length = 90
            feat_size = 600
            np_x_tmp_list = []
            np_y_tmp_list = []
            for i in range(len(df)):
                label = df.at[i, "expr"]
                if label >= 0:         
                    if ignore_feat==True:
                        np_tmp = np.zeros((batch_length, feat_size))
                        #np_tmp2 = df_feat.iloc[i-batch_length+1:i+1, :].values
                        if i-batch_length+1 < 0:
                            np_tmp = np.zeros((batch_length-i-1, feat_size))
                            np_tmp2 = df.iloc[0:i+1, 0:feat_size].values
                            np_tmp = np.append(np_tmp, np_tmp2, axis=0)
                        else:
                            np_tmp = df.iloc[i-batch_length+1:i+1, 0:feat_size].values

                        np_tmp = np_tmp.astype(np.float32)
                        #np_tmp = np_tmp[::5,:]
                        np_tmp = np_tmp[np.newaxis, ::6, :]
                        np_tmp = np.nan_to_num(np_tmp)

                        np_x_tmp_list.append(np_tmp)
                        np_y_tmp_list.append(label)
                    else:
                        if df.at[i,0]!=np.nan:
                            np_tmp = np.zeros((batch_length, feat_size))
                            #np_tmp2 = df_feat.iloc[i-batch_length+1:i+1, :].values
                            if i-batch_length+1 < 0:
                                np_tmp = np.zeros((batch_length-i-1, feat_size))
                                np_tmp2 = df.iloc[0:i+1, 0:feat_size].values
                                np_tmp = np.append(np_tmp, np_tmp2, axis=0)
                            else:
                                np_tmp = df.iloc[i-batch_length+1:i+1, 0:feat_size].values

                            np_tmp = np_tmp.astype(np.float32)
                            #np_tmp = np_tmp[::5,:]
                            np_tmp = np_tmp[np.newaxis, ::6, :]
                            np_tmp = np.nan_to_num(np_tmp)

                            np_x_tmp_list.append(np_tmp)
                            np_y_tmp_list.append(label)

            if len(np_x_tmp_list) > 0:
                np_x_tmp = np.concatenate([x for x in np_x_tmp_list], 0)
                np_y_tmp = np.array(np_y_tmp_list)
                np_x_list.append(np_x_tmp)
                np_y_list.append(np_y_tmp)

        # finish
        np_data_x = np.concatenate([x for x in np_x_list], 0)
        np_data_y = np.concatenate([x for x in np_y_list], 0).reshape(-1,1)
        #np_data_y = np.concatenate([x for x in np_y_list], 0)

        if balance == 1:
            # balance and shuffle
            np_data_x, np_data_y = np_train_balance(np_data_x, np_data_y)
        elif balance == 2:
            # balance and shuffle
            np_data_x, np_data_y = np_val_balance(np_data_x, np_data_y)
        
        np_x_out_list.append(np_data_x)
        np_y_out_list.append(np_data_y)
        
    np_data_x = np.concatenate([x for x in np_x_out_list], 0)
    np_data_y = np.concatenate([x for x in np_y_out_list], 0).reshape(-1,1) 
        
    np_data_y = transformer.transform(np_data_y).toarray()
    
    return np_data_x, np_data_y

In [10]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)

    # input: folder path including original images
    dir_data = dir_root + "\\dataset\\aff2_images\\dataset"
    dir_audio = dir_root + "\\dataset\\aff2_audio\\dataset"

    # input: expr labels
    dir_label_train = dir_root + "\\src\\annotations\\EXPR_Set\\Train_Set"
    dir_label_val = dir_root + "\\src\\annotations\\EXPR_Set\\Validation_Set"
    
    dir_model = dir_root + "\\model_expr\\model_image"
    
    # output: folder path
    dir_out = dir_root + "\\model_expr\\model_mix"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)
    
    model_path_base = "model_modal_multi"
    model_path_base_pre = "model_image_single_pseudo"
    
    # pre_model
    file_pre_model = dir_model + "\\" + model_path_base_pre + "_best.h5"
    base_model = keras.models.load_model(file_pre_model, custom_objects={'f1':f1})
    layer_name = 'vgg_au'
    pre_model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)  
    # *****
    
    train_list = mFILE.search_files(dir_label_train, valid_names=[".txt"], invalid_names=["wuert"], ext=None, recursive=False)    
    val_list = mFILE.search_files(dir_label_val, valid_names=[".txt"], invalid_names=["wuert"], ext=None, recursive=False)
    
    dummy = [[0],[1],[2],[3],[4],[5],[6]]
    transformer = OneHotEncoder().fit(dummy)
    #np_train_y_hot = transformer.transform(np_train_y).toarray()
    #np_val_y_hot = transformer.transform(np_val_y).toarray()
    
    len_train = len(train_list)
    len_val = len(val_list)
    #train_list = train_list[0:20]
    #val_list = val_list[0:10]
    
    np_train_x, np_train_y = data_read_separate2(train_list, dir_data, dir_audio,
                                            pre_model, transformer, balance=1, sep_num=5, ignore_feat=False)
    
    print(np_train_x.shape)
    print(np_train_y.shape)
    print(np_train_x.nbytes, np_train_y.nbytes)
    
    np_val_x, np_val_y = data_read_separate2(val_list, dir_data, dir_audio,
                                            pre_model, transformer, balance=2, sep_num=5, ignore_feat=False)
    
    print(np_val_x.shape)
    print(np_val_y.shape)
    print(np_val_x.nbytes, np_val_y.nbytes)
    

    len_feat = np_val_x.shape[2]
    len_class = np_val_y.shape[1]
    display(len_class)
    
    # search parameter
    score_list = []
    #***
    
    l_units = [64, 128]
    l_drop = [0.3, 0.5]
    l_lr = [1e-4, 1e-3]
    l_batch = [128, 256]
    MAX_COUNT = len(l_units)*len(l_drop)*len(l_lr)*len(l_batch)
    COUNT = 0
    
    for _units in l_units:
        for _drop in l_drop:
            for _lr in l_lr:
                for _batch in l_batch:
                    model_path = dir_out + "\\" + model_path_base + "_{0:02d}.h5".format(COUNT)
                    scores = create_gru_model(np_train_x, np_train_y, np_val_x, np_val_y, len_class,
                                              n_units=_units, drop=_drop, lr=_lr, batch=_batch,
                                              model_path=model_path)
                    param = [COUNT, _units, _drop, _lr, _batch]
                    param.extend(scores)
                    score_list.append(param)
                    COUNT = COUNT + 1
        
        
    # ******************* validation balances frames  ********************
    print(score_list)
    
    df_res = pd.DataFrame(score_list, columns = ["id", "units", "drop", "lr", "batch", "score", "f1", "acc"])
    display(df_res)
    file_out = dir_out + "\\res0_" + model_path_base + ".csv"
    df_res.to_csv(file_out, index=False)
    
    best_id = df_res.loc[:,"score"].idxmax()
    
    best_path = dir_out + "\\" + model_path_base + "_{0:02d}.h5".format(best_id)
    out_path = dir_out + "\\" + model_path_base + "_best.h5"
    #best_path = dir_out + "\\model_image_multi_b2" + "_{0:02d}.h5".format(best_id)
    #out_path = dir_out + "\\model_image_multi_b2_best.h5"
    shutil.copy(best_path, out_path)
    
    
    np_x_list = None
    np_y_list = None
    np_train_x = None
    np_train_y = None
    np_val_x = None
    np_val_y = None
    
    # ******************* validation all frames  ********************
    
    #file_model = dir_out + "\\model_image_multi_best.h5"
    model = keras.models.load_model(out_path, custom_objects={'f1':f1})
    model.summary()
    # ******
    
    np_val_x, np_val_y = data_read_separate2(val_list, dir_data, dir_audio,
                                            pre_model, transformer, balance=0, sep_num=5, ignore_feat=True)
    
    print(np_val_x.shape)
    print(np_val_y.shape)
    print(np_val_x.nbytes, np_val_y.nbytes)

    len_feat = np_val_x.shape[2]
    len_class = np_val_y.shape[1]
    display(len_class)
    
    pred_nn = model.predict(np_val_x)
    score_nn_f = f1_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1), average='macro')
    score_nn_a = accuracy_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1))
    score_nn = score_nn_f*0.67 + score_nn_a*0.33
    
    scores_nn = [score_nn, score_nn_f, score_nn_a]
    display(["0.67*F1+0.33*ACC", "F1 score", "ACC score"])
    display(scores_nn)
    
    df_out = eval_pred_each_class(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1), 7)
    display(df_out)
    
    file_out = dir_out + "\\res1_" + model_path_base + ".csv"
    df_out.to_csv(file_out, index=False)
    

In [11]:
if __name__ == "__main__":
    main()

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

(327818, 10, 600)
(327818, 7)
7867632000 18357808


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:45<00:00,  3.25s/it]


(88136, 10, 600)
(88136, 7)
2115264000 4935616


7

...

(243006, 10, 600)
(243006, 7)
5832144000 13608336


7

['0.67*F1+0.33*ACC', 'F1 score', 'ACC score']

[0.5105859788690337, 0.46471720544120465, 0.6037134885558382]

,class,recall,precision,f1,accuracy
0,0,0.741926,0.671588,0.705007,0.749681
1,1,0.528605,0.307637,0.388927,0.944680
2,2,0.262444,0.235630,0.248315,0.961906
3,3,0.182117,0.474304,0.263181,0.959170
4,4,0.661021,0.671147,0.666045,0.849576
5,5,0.343874,0.597304,0.436468,0.854049
6,6,0.617994,0.487550,0.545076,0.888365
7,all,0.476854,0.492166,0.464717,0.886775
